In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import MBartForConditionalGeneration, MBartTokenizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from rouge_score import rouge_scorer

c:\Anaconda\envs\Cuda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Anaconda\envs\Cuda\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Anaconda\envs\Cuda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Anaconda\envs\Cuda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
DF = pd.read_json('../Datasets/AIC Val/ds.jsonl', lines=True)
DF.head()
# DF = pd.read_csv('../Datasets\WikiHow\wikiHow.csv', nrows=2000)
# DF.columns = ['summary', 'paragraph']
# DF

,example_id,paragraph,summary
0,0,وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب ع...,يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الك...
1,1,ولم يعترف دبلوماسيو هاتين الدولتين بالعريضة ال...,دبلوماسيو الدولتين لم يعترفوا بالعريضة التي قا...
2,2,قامت ولاية حلب بعد اعلان الجنرال الفرنسي هنري ...,أعلن غورو الانتداب الفرنسي على سوريا لكي يعاقب...
3,3,دولة مصر العربيه هي ليست اي دوله وليست اي شعب ...,مصر هي أم البلاد، وقائدة العرب؛ فهي أرض بلاد ا...
4,4,السوريون يصرون على استقلال بلادهم : و مثلما رف...,الشعب السوري يصر على استقلال بلدهم من السيطرة ...


In [4]:
Paragraphs = DF['paragraph'].tolist()
Summaries = DF['summary'].tolist()

In [5]:
Train_Paragraphs, Test_Paragraphs, Train_Summaries, Test_Summaries = train_test_split(Paragraphs, Summaries, test_size=0.2, random_state=42)
Train_Paragraphs, Validation_Paragraphs, Train_Summaries, Validation_Summaries = train_test_split(Train_Paragraphs, Train_Summaries, test_size=0.2, random_state=42)

In [6]:
Model_Name = "facebook/mbart-large-50"
Tokenizer = MBartTokenizer.from_pretrained(Model_Name)
Model = MBartForConditionalGeneration.from_pretrained(Model_Name).to(device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
Paragraph_Max_Length = 512
Train_Paragraph_Encodings = Tokenizer(Train_Paragraphs, truncation=True, max_length=Paragraph_Max_Length, padding="max_length", return_tensors="pt").to(device)
Validation_Paragraph_Encodings = Tokenizer(Validation_Paragraphs, truncation=True, max_length=Paragraph_Max_Length, padding="max_length", return_tensors="pt").to(device)
Test_Paragraph_Encodings = Tokenizer(Test_Paragraphs, truncation=True, max_length=Paragraph_Max_Length, padding="max_length", return_tensors="pt").to(device)

Summary_Max_Length = 64
Train_Summary_Encodings = Tokenizer(Train_Summaries, truncation=True, max_length=Summary_Max_Length, padding="max_length", return_tensors="pt").to(device)
Validation_Summary_Encodings = Tokenizer(Validation_Summaries, truncation=True, max_length=Summary_Max_Length, padding="max_length", return_tensors="pt").to(device)
Test_Summary_Encodings = Tokenizer(Test_Summaries, truncation=True, max_length=Summary_Max_Length, padding="max_length", return_tensors="pt").to(device)

In [8]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_mask, decoder_input_ids, decoder_attention_mask):
        self.input_ids = input_ids.to(device)
        self.attention_mask = attention_mask.to(device)
        self.decoder_input_ids = decoder_input_ids.to(device)
        self.decoder_attention_mask = decoder_attention_mask.to(device)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "decoder_input_ids": self.decoder_input_ids[idx],
            "decoder_attention_mask": self.decoder_attention_mask[idx]
        }

In [9]:
Train_Dataset = CustomDataset(
    Train_Paragraph_Encodings['input_ids'],
    Train_Paragraph_Encodings['attention_mask'],
    Train_Summary_Encodings['input_ids'],
    Train_Summary_Encodings['attention_mask']
)

Validation_Dataset = CustomDataset(
    Validation_Paragraph_Encodings['input_ids'],
    Validation_Paragraph_Encodings['attention_mask'],
    Validation_Summary_Encodings['input_ids'],
    Validation_Summary_Encodings['attention_mask']
)

Test_Dataset = CustomDataset(
    Test_Paragraph_Encodings['input_ids'],
    Test_Paragraph_Encodings['attention_mask'],
    Test_Summary_Encodings['input_ids'],
    Test_Summary_Encodings['attention_mask']
)

In [10]:
Train_Dataloader = DataLoader(Train_Dataset, batch_size=1)
Validation_Dataloader = DataLoader(Validation_Dataset, batch_size=1)
Test_Dataloader = DataLoader(Test_Dataset, batch_size=1)

In [11]:
Scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])

In [12]:
Optimizer = torch.optim.AdamW(Model.parameters(), lr=1e-5)
Criterion = torch.nn.CrossEntropyLoss().to(device)

Epochs_Number = 5

for Epoch in range(Epochs_Number):
    Model.train()
    Total_Loss = 0

    Total_Rouge1 = 0.0
    Total_Rouge2 = 0.0
    Total_RougeL = 0.0
    Total_Batches = 0
    with tqdm(Train_Dataloader, desc=f"Epoch {Epoch + 1}/{Epochs_Number}",  unit="batch") as t:
        for Batch in t:
            Input_IDs = Batch["input_ids"].to(device)
            Attention_Mask = Batch["attention_mask"].to(device)
            Decoder_Input_IDs = Batch["decoder_input_ids"].to(device)
            Decoder_Attention_Mask = Batch["decoder_attention_mask"].to(device)

            # print("Input IDs Shape:", Input_IDs.shape)
            # print("Attention Mask Shape:", Attention_Mask.shape)
            # print("Decoder Input IDs Shape:", Decoder_Input_IDs.shape)
            # print("Decoder Attention Mask Shape:", Decoder_Attention_Mask.shape)

            Outputs = Model(
                input_ids=Input_IDs,
                attention_mask=Attention_Mask,
                decoder_input_ids=Decoder_Input_IDs,
                decoder_attention_mask=Decoder_Attention_Mask,
                labels=Decoder_Input_IDs
            )

            # IDs = Tokenizer.batch_decode(Input_IDs, skip_special_tokens=True)
            # decoded_output = Tokenizer.batch_decode(Outputs.logits.argmax(dim=-1), skip_special_tokens=True)
            # print(IDs)
            # print(decoded_output)

            Loss = Outputs.loss
            Total_Loss += Loss.item()

            Optimizer.zero_grad()
            Loss.backward()
            Optimizer.step()

            Reference_sentences = Tokenizer.batch_decode(Decoder_Input_IDs, skip_special_tokens=True)
            Generated_sentences = Tokenizer.batch_decode(Outputs.logits.argmax(dim=-1), skip_special_tokens=True)

            Scores = Scorer.score(''.join(Reference_sentences), ''.join(Generated_sentences))
            Total_Rouge1 += Scores['rouge1'].fmeasure
            Total_Rouge2 += Scores['rouge2'].fmeasure
            Total_RougeL += Scores['rougeL'].fmeasure
            Total_Batches += 1

            t.set_postfix(
                Loss_Average=Total_Loss / len(Train_Dataloader),
                loss=Loss.item(),
                Rouge1_Average=Total_Rouge1 / Total_Batches,
                Rouge2_Average=Total_Rouge2 / Total_Batches,
                RougeL_Average=Total_RougeL / Total_Batches
            )

Epoch 1/5:   0%|          | 0/98 [00:00<?, ?batch/s]

Epoch 5/5: 100%|██████████| 98/98 [07:28<00:00,  4.57s/batch, Loss_Average=0.0047, Rouge1_Average=0.194, Rouge2_Average=0.0408, RougeL_Average=0.194, loss=0.00215]  


In [12]:
Model_Save_Path = 'mBart.pt'

In [ ]:
torch.save(Model.state_dict(), Model_Save_Path)

In [13]:
Model.load_state_dict(torch.load(Model_Save_Path, map_location=torch.device(device)))

<All keys matched successfully>

In [22]:
Test_Paragraph_Encodings['input_ids']

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')

In [26]:
len(Test_Paragraph_Encodings['input_ids'])

31

In [25]:
Total_Rouge1 = 0.0
Total_Rouge2 = 0.0
Total_RougeL = 0.0
Total_Batches = 0
with tqdm(Test_Dataloader, desc="Testing: ",  unit="batch") as t:
    for Batch in t:
        Input_IDs = Batch["input_ids"].to(device)
        Attention_Mask = Batch["attention_mask"].to(device)
        Decoder_Input_IDs = Batch["decoder_input_ids"].to(device)
        Decoder_Attention_Mask = Batch["decoder_attention_mask"].to(device)

        # print("Input IDs Shape:", Input_IDs.shape)
        # print("Attention Mask Shape:", Attention_Mask.shape)
        # print("Decoder Input IDs Shape:", Decoder_Input_IDs.shape)
        # print("Decoder Attention Mask Shape:", Decoder_Attention_Mask.shape)

        Outputs = Model(
            input_ids=Input_IDs,
            attention_mask=Attention_Mask,
            labels=Decoder_Input_IDs
        )

        IDs = Tokenizer.batch_decode(Input_IDs, skip_special_tokens=True)
        Decoded_Output = Tokenizer.batch_decode(Outputs.logits.argmax(dim=-1), skip_special_tokens=True)
        print(IDs)
        print(Decoded_Output)

        Loss = Outputs.loss
        Total_Loss += Loss.item()

        Loss.backward()

        Reference_sentences = Tokenizer.batch_decode(Decoder_Input_IDs, skip_special_tokens=True)
        Generated_sentences = Tokenizer.batch_decode(Outputs.logits.argmax(dim=-1), skip_special_tokens=True)

        Scores = Scorer.score(''.join(Reference_sentences), ''.join(Generated_sentences))
        Total_Rouge1 += Scores['rouge1'].fmeasure
        Total_Rouge2 += Scores['rouge2'].fmeasure
        Total_RougeL += Scores['rougeL'].fmeasure
        Total_Batches += 1

        t.set_postfix(
            Loss_Average=Total_Loss / len(Train_Dataloader),
            loss=Loss.item(),
            Rouge1_Average=Total_Rouge1 / Total_Batches,
            Rouge2_Average=Total_Rouge2 / Total_Batches,
            RougeL_Average=Total_RougeL / Total_Batches
        )

Testing:   0%|          | 0/31 [00:00<?, ?batch/s]

['اعلنت " لجنة جائزة بشارة الخوري للتوعية الديموقراطية " انها انجزت المسابقة التي نظمتها حول " دور الرئيس بشارة الخوري في تحقيق الميثاق والاستقلال " بالتعاون مع وزارة التربية والتعليم العالي وبدعم من " مؤسسة فريديريتش ايبرت "، والتي شارك فيها طلاب الصف الثانوي الثاني بفرعيه العلمي والادبي والذين ينتمون الى مدارس رسمية وخاصة في المحافظات اللبنانية ، لافتة الى ان اعلان النتائج وتوزيع الجوائز سيتم خلال احتفال يقام عند الخامسة من بعد ظهر الخميس 30 الحالي في قصر الاونيسكو. واوضحت اللجنة في بيان " ان المسابقة تناولت تاريخ لبنان الحديث من قيام دولة لبنان الكبير في الاول من ايلول 1920، الى الجلاء في 31 كانون الاول 1946، دور الرئيس بشارة الخوري في تحقيق الوحدة الوطنية واهمية الالتزام بالميثاق الوطني للمحافظة على الاستقلال ". واشارت " الى ان الطلاب المشاركين اخضعوا في مرحلة اولى لامتحان خطي فاز في نهايته 69 طالبا ، ثم اشترك 17 منهم ممن نالوا العلامات الاعلى في المرحلة الثانية من المسابقة التي تضمنت امتحانا شفهيا امام لجنة فاحصة من الاساتذة المختصين اجابوا فيه على اسئلة تناولت فترة ولادة الاستقلا

Testing:   3%|▎         | 1/31 [00:04<02:18,  4.63s/batch, Loss_Average=9.67, Rouge1_Average=0, Rouge2_Average=0, RougeL_Average=0, loss=22.7]

['و لما استولى العباسيون على زمام الأمور أهمل شأن البريد إلى زمن المهدي حين أغزى ابنه هارون الرشيد بلاد الروم ، فرتب بينه وبين معسكر ابنه بردا تأتيه بأخباره ، ولما عاد الرشيد من مهمته توقفت تلك البرد ، وحين تولى الرشيد الخلافة ذكر ما كان من أبيه بشأن البريد فأمر بتنظيمه وقرره على نحو ما كان أيام بني أمية ، وزاد خلفاؤه فيه حتى غدا أشبه بدائرة للاستخبارات والخدمات العاجلة ، وأقيمت محطات البريد في كل أرجاء الخلافة. واستمر أمر البريد على هذا النحو إلى أن غلب بنو بويه على الخلافة فقطعوا البريد ليخفى على الخليفة ما يكون من أخبارهم ، وتابعهم السلاجقة في هذا الأمر ، فلم يكن بين ملوك الإسلام إلا الرسل على الدواب. وفي العهد الزنكي وضع للبريد نظام جديد يقوم على خدمته النجابة ( البريدية ) وخصصت لهم النجائب المنتخبة ، ودام ذلك حتى نهاية الدولة الأيوبية وصعود دولة المماليك. وفي العهد الزنكي أيضا كان ابتداء استخدام الطيور الهوادي ( حمام الزاجل ) في نقل البريد. وكان السلطان الملك العادل نور الدين بن زنكي أول من استخدمه من ملوك الإسلام في نقل الرسائل بانتظام ابتداء من سنة 565 ه. وأقام له مطارات وأبراج 

Testing:   6%|▋         | 2/31 [00:09<02:24,  5.00s/batch, Loss_Average=9.93, Rouge1_Average=0, Rouge2_Average=0, RougeL_Average=0, loss=25]  

...

['كان لشيوخ الأزهر دور كبير في تنصيب محمد علي واليا على مصر ، فعلماء الأزهر المنتشون بدورهم السياسي الكبير في زمن الحقبة الفرنسية ، شاركوا بقوة في الثورة والاحتجاج على الوالي العثماني أحمد خورشيد باشا ، وطالبوا الباب العالي بتعيين محمد علي بديلا عنه. ولكن بمجرد وصول الداهية الألباني محمد علي إلى كرسي الولاية ، أزاح هؤلاء العلماء جانبا ، حتى يخلو له حكم الأراضي المصرية ، فعمل على ضربهم بعضهم ببعض ، ونسج خيوط مؤامرة لنفي السيد عمر مكرم نقيب الأشراف الأزهري إلى دمياط ، وجرده من ألقابه ومناصبه ، ومن المؤسف أن عددا من كبار العلماء قد اشتركوا في تلك المؤامرة مثل الشيخ عبد الله الشرقاوي والشيخ محمد السادات. ثم وجه محمد علي ضربته الكبرى للأزهر ونفوذه عندما وضع يده على أملاك الأوقاف ، ووضع يده كذلك على كل الأراضي الزراعية في القطر المصري ، وأصبح بذلك مالك كل شيء في مصر. في عهد خلفاء محمد علي الأوائل ، اعتاد رجالات الأزهر على مسايرة الحكام والتوافق معهم ، خوفا من أن يتم تجريدهم من البقية الباقية من نفوذهم الروحي في البلاد ، ولكن ذلك لم يمنع من حدوث بعض الخلافات والمصادمات التي انتشرت وذاعت في كت

Testing:  26%|██▌       | 8/31 [00:41<02:05,  5.46s/batch, Loss_Average=11.4, Rouge1_Average=0.125, Rouge2_Average=0, RougeL_Average=0.125, loss=24.5]

['لعبت الحضارة الفينيقية دورا مهما في اكتشاف الأبجدية ونقلها إلى العالم. وعندما نذكر صيدون الفينيقية لا بد أن نذكر الإنجاز الحضاري العظيم الذي قدمته هذه المدينة إلى بلاد اليونان ألا وهو نشر الأبجدية فيها. وهذا ما تؤكده روايات مؤرخي اليونان الذين يذكرون أنهم عرفوا الحروف الهجائية عن طريق الصيدونيين الذين جاءوا إلى بلاد اليونان حوالي سنة 1580 ق. م. صحبة قدموس الذي حمل معه الحروف الهجائية وبنى مدينة تيبة وتملكها. كما كان للحضارة الفينيقية دور مهم في اكتشاف مادة الصباغ الأرجواني وتصديرها إلى العالم ، ( مادة الصباغ الأرجواني موجودة في حيوانات بحرية ذات أصداف تسمى الموريكس وكان لونها أحمر بنفسجيا ، ويتم استعمال هذه المادة في صباغ الحرير والقطن والصوف الناعم ) ويذهب البعض إلى أن مدينة صيدون هي مكتشفة الصباغ الأرجواني بخلاف ما ذكر من أن صور هي مكتشفته ( وجد جبل كامل من هذه الأصداف عند مقام أبا روح على شاطئ صيدا الجنوبي ويعود تاريخها إلى أوائل الألف الثاني قبل الميلاد في حين أن آثار مصانع الأرجوان حول مدينة صور تعود إلى القرن الثاني عشر قبل الميلاد فقط ). والصيدونيون هم أول من اصطنع الزجاج ولا 

Testing:  26%|██▌       | 8/31 [00:47<02:16,  5.92s/batch, Loss_Average=11.4, Rouge1_Average=0.125, Rouge2_Average=0, RougeL_Average=0.125, loss=24.5]


KeyboardInterrupt: 

In [19]:
Model.eval()
Total_Loss = 0

Total_Rouge1 = 0.0
Total_Rouge2 = 0.0
Total_RougeL = 0.0
Total_Batches = 0

with torch.no_grad():
    with tqdm(Train_Dataloader, desc="Testing: ",  unit="batch") as t:
        for Batch in t:
            Input_IDs = Batch["input_ids"].to(device)
            Attention_Mask = Batch["attention_mask"].to(device)
            Decoder_Input_IDs = Batch["decoder_input_ids"].to(device)
            Decoder_Attention_Mask = Batch["decoder_attention_mask"].to(device)

            Outputs = Model(
                input_ids=Input_IDs,
                attention_mask=Attention_Mask
            )

            # Loss = Outputs.loss
            # Total_Loss += Loss.item()
            
            IDs = Tokenizer.batch_decode(Input_IDs, skip_special_tokens=True)
            Decoded_Output = Tokenizer.batch_decode(Outputs.logits.argmax(dim=-1), skip_special_tokens=True)
            print(IDs)
            print(Decoded_Output)

            Reference_sentences = Tokenizer.batch_decode(Decoder_Input_IDs, skip_special_tokens=True)
            Generated_sentences = Tokenizer.batch_decode(Outputs.logits.argmax(dim=-1), skip_special_tokens=True)

            Scores = Scorer.score(''.join(Reference_sentences), ''.join(Generated_sentences))
            Total_Rouge1 += Scores['rouge1'].fmeasure
            Total_Rouge2 += Scores['rouge2'].fmeasure
            Total_RougeL += Scores['rougeL'].fmeasure
            Total_Batches += 1

            t.set_postfix(
                # Loss_Average=Total_Loss / len(Train_Dataloader),
                # loss=Loss.item(),
                Rouge1_Average=Total_Rouge1 / Total_Batches,
                Rouge2_Average=Total_Rouge2 / Total_Batches,
                RougeL_Average=Total_RougeL / Total_Batches
            )



Testing:   0%|          | 0/98 [00:00<?, ?batch/s]

Testing:   1%|          | 1/98 [00:00<01:32,  1.05batch/s, Rouge1_Average=0, Rouge2_Average=0, RougeL_Average=0]

['التاريخ يقول ان غزو " ا آشوريا " لمصر حصل في القرن السابع قبل الميلاد واحتلت على اثره مصر لمدة تقارب المائة سنة تخللها مقاومة وحروب وقلاقل وثورات من المصريين لمقاومة الاحتلال الآشوري وفي النهاية استطاع المصريون طرد الآشوريين الغزاة لكن ما لبثوا ان غزاهم الفرس في منتصف القرن السادس قبل الميلاد وسقطت مصر في أيدي الفرس ورزحت تحت الاحتلال الفارسي قرابة 180 سنة بعد ان احتلها الآشوريون لمدة تقارب قرنا من الزمان ثم جاء اليونانيون بقيادة الإسكندر الأكبر المقدوني واحتل مصر وانهى الحقبة الفارسية في مصر واستمر حكم البطالمة لمصر قرابة ثلاثة قرون وما كادت مصر تخرج من نفوذ الحكم اليوناني حتى وقعت تحت الحكم الروماني واسمر حتى سقط على يد عمرو بن العاص فدخلت مصر تحت حكم العرب الى اليوم وان كان تخلله فترات حكم مصر خلالها المماليك والاخشيديون وأخيرا العثمانيون الأتراك وخضعت فترة قصيرة للانتداب البريطاني حتى نالت استقلالها وتأسست جمهورية مصر الحديثة واذا استثنينا فترة الانتداب البريطاني فان كل الحكومات من بعد الفتح الاسلامي لمصر حكومات إسلامية.. ومنذ الفتح الاسلامي لمصر توافد العرب زرافات الى مصر من جزي

Testing:   3%|▎         | 3/98 [00:02<01:13,  1.30batch/s, Rouge1_Average=0, Rouge2_Average=0, RougeL_Average=0]

['تقع مدينة تدمر على بعد 160 كم شرقي حمص وسط واحة في بادية الشام ، ترويها ينابيع كبريتية ، وكان أول من سكنها الكنعانيون الذين عبدوا فيها إلههم ( بعل )، وكان ذلك في الألف الثالثة قبل الميلاد ، ثم استوطنها الآراميون بعدهم وقد أعطاها الآراميون اسمها ( تدمرتو ) أي الأعجوبة, أما اليونانيون و الرومان فسموها ( بالميرا ) نسبة إلى النخيل. وفي القرن الخامس قبل الميلاد أخضعها الفرس لحكمهم ، ثم استولى عليها الإسكندر الكبير المقدوني ، وكانت بعد موته من نصيب السلوقيين في سورية ، ولكنها بقيت بعيدة عن سيطرتهم السياسية الكاملة ، ومع ذلك تأثرت بالفن الهلنسي. وعندما فتح الرومان سورية عام 64 ق. م ، ظلت تدمر خارج سيطرتهم ، حتى عهد الإمبراطور ( تيبرسوس 14-27 م )، عندما منحها الرومان لقب " المدينة الحرة "، ثم حازت لقب ( مستعمرة رومانية )، وبموجب ذلك أعفيت من الضرائب ، ومنحت امتيازات كثيرة مستفيدة من وجود أباطرة عرب حكموا روما في هذه الفترة. وقد آلت تدمر إلى الهيمنة الرومانية عام 160 م ، فعمل جنود الهجانة من تدمر في الجيش الروماني لحمايتها. وعندما استولى الفرس على سوريا في حوالي عام 260 م لم يكن لدى روما جنود

...

['أما بخصوص التنظيم والممارسات الإدارية المحلية والمركزية فإنها هي التي أكسبت مصر منذ بداية حكم أغسطس طابع الولاية الرومانية. وكان على مصر في ظل التنظيم الإمبراطوري الكبير أن تزود روما بثلث احتياجاتها السنوية من الحبوب اللازمة لإطعام الشعب الروماني. ولكي يضمن أغسطس عدم تمزق مصر أو انحرافها عن الهدف السابق ( تزويد روما بثلث ما تحتاج إليه سنويا من الحبوب ) فقد جعل منها ولاية أشبه ما تكون بضيعة خاصة بالإمبراطور. فعلى النقيض من الولايات الأخرى التي كان يتولى الحكم فيها رومان تبوأوا مكانا عليا وارتقوا في سلم الإدارة الرومانية حتى وظيفة قناصل ثم تركوا القنصلية وأصبحوا قناصل سابقين proconsul فإن حكم مصر أسند إلى موظفين أقل يطلق عليهم لقب الوالي praefectus وكان يعتبر قائما بالأعمال يعينه الإمبراطور كممثل شخصي له. وجرى العرف منذ عهد أغسطس على أنه يكون والي مصر من طبقة الفرسان وهي الطبقة التي شكلت منذ بداية حكم الإمبراطور أغسطس العمود الفقري والمتين والحصن الصلب في تأييده. وقد اشترط أغسطس ألا يسمح للرومان من طبقة السناتو أو حتى من الشخصيات البارزة العامة من طبقة الفرسان بدخول مصر إلا بعد الحصول 

Testing:  32%|███▏      | 31/98 [00:25<00:50,  1.33batch/s, Rouge1_Average=0.0558, Rouge2_Average=0.0108, RougeL_Average=0.0558]

['أخيرا في 28 سبتمبر 1918 خرج العثمانيون من بلاد الشام ، ورفع العلم العربي فوق سراي بيروت وبعبدا ثم أنزل في 6 أكتوبر واستبدل بعلم فرنسا بعد احتلال جيوشها لساحل بلاد الشام. حاولت فرنسا عن طريق مراسلات بين بكركي وجورج بيكو إقناع البطريرك بمحاسن الإبقاء على الوحدة مع سوريا ، لكن جواب البطريرك كان حازما بطلب الاستقلال الكامل ؛ وقد أفضى مؤتمر فرساي في باريس إلى تشكيل لجنة كينغ كراين لاستقصاء رغبات سكان بلاد الشام ، وقد قابلت اللجنة الحويك في يونيو 1918 وقدمت توصياتها باستقلال لبنان ، وذلك حافظا على رغبة أبنائه من صور إلى طرابلس. في أغسطس سافر الحويك على رأس وفد لبناني لحضور الجلسة الثانية لمؤتمر الصلح ، ولقب حينها بطريرك لبنان للإشارة إلى التفاف اللبنانيين مسلمين ومسيحيين حوله ، وخلال زيارته طالب البطريرك مجددا استقلال لبنان. مكث الوفد في باريس أربع أشهر ، وعندما عاد إلى بيروت كشف البطريرك عن رسالة من رئيس وزراء فرنسا جورج كليمنصو مؤرخة في 5 يناير 1920 تفيد بدعم فرنسا استقلال لبنان ثم قبل فيصل الأول ملك المملكة السورية العربية استقلال لبنان بموجب اتفاق فيصل كلمنصو المؤرخ في 6 يناير 1920، وخ

Testing:  33%|███▎      | 32/98 [00:26<00:54,  1.20batch/s, Rouge1_Average=0.0697, Rouge2_Average=0.0104, RougeL_Average=0.0697]


KeyboardInterrupt: 

In [29]:
# Model.eval()

# Total_Loss = 0
# Total_Rouge1 = 0.0
# Total_Rouge2 = 0.0
# Total_RougeL = 0.0
# Total_Batches = 0

# with torch.no_grad():
#     with tqdm(Test_Dataloader, desc="Testing", unit="batch") as t:
#         for Batch in t:
#             Input_IDs = Batch["input_ids"].to(device)
#             Attention_Mask = Batch["attention_mask"].to(device)
#             Decoder_Input_IDs = Batch["decoder_input_ids"].to(device)
#             Decoder_Attention_Mask = Batch["decoder_attention_mask"].to(device)

#             print(Input_IDs)
#             print(len(Input_IDs[0]))
#             Outputs = Model.generate(
#                 input_ids=Input_IDs,
#                 attention_mask=Attention_Mask,
#                 max_length=100,
#                 num_beams=4,
#                 early_stopping=True,
#                 return_dict_in_generate=True
#             )
            
#             IDs = Tokenizer.batch_decode(Input_IDs, skip_special_tokens=True)
#             Decoded_Output = Tokenizer.batch_decode(Outputs.sequences, skip_special_tokens=True)
#             print(IDs)
#             print(Decoded_Output)

#             # Loss = Outputs.loss
#             # Total_Loss += Loss.item()

#             Reference_sentences = Tokenizer.batch_decode(Decoder_Input_IDs, skip_special_tokens=True)
#             Generated_sentences = Tokenizer.batch_decode(Outputs.sequences, skip_special_tokens=True)

#             Scores = Scorer.score(''.join(Reference_sentences), ''.join(Generated_sentences))
#             Total_Rouge1 += Scores['rouge1'].fmeasure
#             Total_Rouge2 += Scores['rouge2'].fmeasure
#             Total_RougeL += Scores['rougeL'].fmeasure
#             Total_Batches += 1

#             t.set_postfix(
#                 # Loss_Average=Total_Loss / len(Test_Dataloader),
#                 # loss=Loss.item(),
#                 Rouge1_Average=Total_Rouge1 / Total_Batches,
#                 Rouge2_Average=Total_Rouge2 / Total_Batches,
#                 RougeL_Average=Total_RougeL / Total_Batches
#             )
#             break

Testing:   0%|          | 0/31 [00:00<?, ?batch/s]

tensor([[   878,  65306,    368,     44,  87202,      6, 102002, 133017,    250,
          10754,  25288,  28771, 158219, 108451,  15617, 167591,    648,     44,
          91587,    716,  29602,    368, 233090,   1761,  61818,   6466,  14626,
             44,   6501,  24094, 133017,    250,  10754,  25288,    240,  35394,
            906,  86765,   6472,   1101, 192761,     44, 185492,   1546,  38126,
          93259, 137811, 115537,   6268,  54067,    230,     44, 135057, 192778,
          53314,  45802,   5669, 136549,     44,     50,  26426, 131955,  13061,
         168461,  54191,  23372,   9793,    179,  27944, 167315,  15814,    176,
         122129,   1101,   6227,  12337,  44236,    323,    767,  12371,  23090,
           6449,  73703,  74023,    250, 140588,    240,    359, 164220, 179846,
            725,   1327,  10488,    250,   6449,    716,  14848, 154574,   3426,
         105268,   1195,  51416,  48236, 111640,   5784, 232066,    767,  32848,
          11020, 200561,    

Testing:   0%|          | 0/31 [00:02<?, ?batch/s]


AttributeError: 'Seq2SeqLMOutput' object has no attribute 'sequences'

In [ ]:
paragraph = "علمت الأنباء بأنه تم إعلان نتائج الانتخابات الأخيرة في البلاد. حظي المرشح الأول بدعم واسع من الناخبين وفاز بالمنصب الرئاسي. وتم تكليفه بتشكيل حكومة جديدة للبلاد. يأمل الناس أن تسهم هذه الحكومة في تحقيق التقدم والاستقرار ومعالجة التحديات التي تواجه البلاد. من المتوقع أن تكون السنوات القادمة حافلة بالتحولات والتغيرات الإيجابية. نأمل أن يكون للبلاد مستقبل مشرق وأن تزدهر الأمور في جميع المجالات."

input_tokens = Tokenizer.encode_plus(paragraph, add_special_tokens=True, truncation=True, max_length=512, padding='longest', return_tensors='pt')
input_ids = input_tokens['input_ids'].to(device)
attention_mask = input_tokens['attention_mask'].to(device)

# Pass the input tensors through the model
Model.eval()
with torch.no_grad():
    outputs = Model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=100, num_beams=4, early_stopping=True)

# Decode the predicted summary
summary = Tokenizer.decode(outputs[0], skip_special_tokens=True)
print(len(paragraph))
print(paragraph)
print(summary)
print(len(summary))

In [ ]:
# Model.eval()

# Predicted_Summaries = []

# with tqdm(Train_Dataloader, unit="batch") as t:
#     for Batch in t:
#         Input_IDs = Batch["input_ids"]
#         Attention_Mask = Batch["attention_mask"]
#         Outputs = Model.generate(
#             input_ids=Input_IDs,
#             attention_mask=Attention_Mask,
#             max_length=Summary_Max_Length,
#             num_beams=4,
#             length_penalty=2.0,
#             early_stopping=True
#         )
         
#         print(Outputs)
#         IDs = Tokenizer.batch_decode(Input_IDs, skip_special_tokens=True)
#         print(IDs)
#         Batch_Summaries = Tokenizer.batch_decode(Outputs, skip_special_tokens=True)
#         print(Batch_Summaries)
#         Predicted_Summaries.extend(Batch_Summaries)


In [ ]:
# Train_Paragraphs[0]

'التاريخ يقول ان غزو " ا آشوريا " لمصر حصل في القرن السابع قبل الميلاد واحتلت على اثره مصر لمدة تقارب المائة سنة تخللها مقاومة وحروب وقلاقل وثورات من المصريين لمقاومة الاحتلال الآشوري وفي النهاية استطاع المصريون طرد الآشوريين الغزاة لكن ما لبثوا ان غزاهم الفرس في منتصف القرن السادس قبل الميلاد وسقطت مصر في أيدي الفرس ورزحت تحت الاحتلال الفارسي قرابة 180 سنة بعد ان احتلها الآشوريون لمدة تقارب قرنا من الزمان ثم جاء اليونانيون بقيادة الإسكندر الأكبر المقدوني واحتل مصر وانهى الحقبة الفارسية في مصر واستمر حكم البطالمة لمصر قرابة ثلاثة قرون وما كادت مصر تخرج من نفوذ الحكم اليوناني حتى وقعت تحت الحكم الروماني واسمر حتى سقط على يد عمرو بن العاص فدخلت مصر تحت حكم العرب الى اليوم وان كان تخلله فترات حكم مصر خلالها المماليك والاخشيديون وأخيرا العثمانيون الأتراك وخضعت فترة قصيرة للانتداب البريطاني حتى نالت استقلالها وتأسست جمهورية مصر الحديثة واذا استثنينا فترة الانتداب البريطاني فان كل الحكومات من بعد الفتح الاسلامي لمصر حكومات إسلامية .. ومنذ الفتح الاسلامي لمصر توافد العرب زرافات الى مصر من جزي

In [ ]:
# print(Model)
# print(type(Model))

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm)

In [ ]:
# text = 'تاريخ أقول غزو " آشوريا " مصر حصل قرن سابع ميلاد احتال ثرا مصر مد تقارب المائة تخلل مقاوم حروب وقلاقل ثور مصري مقاوم احتلال الآشوري نهاية استطاع مصري طرد الآشوريين غزاة لبث غز فرس منتصف قرن سادس ميلاد سقط مصر يد فرس رزح احتلال فارس قراب ١٨٠ احتل الآشوريون مد تقارب قرن زم جاء يوناني قياد الإسكندر أكبر مقدوني حتل مصر وانهى حقب فارس مصر استمر حكم البطالمة مصر قراب قرون ما كاد مصر تخرج نفوذ حكم يونان وقع حكم رومان سمر سقط يد عمر العاص دخل مصر حكم عرب تخلل فتر حكم مصر خلال مماليك والاخشيديون أخير عثماني الأتراك خضع فترة قصير انتداب بريطاني نال استقلال تأسس جمهور مصر حديث واذا استثنى فترة انتداب بريطاني حكومة فتح الاسلامي مصر حكومة إسلام منذ فتح الاسلامي مصر توافد عرب زرافة مصر جزيرة عرب لاحظ تاريخ مصر سياسة حضار مدون بدأ تولي الاسرة فرعون ٣٥٠٠ ميلاد تعدي قدم حضار عالم تدوين حكام محكوم وقت افارقة محلي نقول عن اصحاب سلال جين e سلال جين لافارقة جنوب صحراء ( أمازيغ نوبي حبش ) انتهى حكم مصر خرج يد حكام محلي دخول الإسكندر أكبر مقدوني مصر ظن سلال جين j٢ دخل مصر قدوم الآشوريين فرس مصر دخل سلال جين j١ دخول الآشوريين لان الساميين اختلط الاشوريين الامبراطورية الآشورية كانو مكون اجتماع إمبراطوري الاشورية بابلي سومر عراق والشام ازداد ضخ سلال جين j١ مصر فتح الاسلامي مصر استنتج سلال جين مصري أصلي e سلال جين j١ j٢ وافد أجانب مصر ذلك جبى نسى سلال جين الهندواوروبية r دخل مصر دخول البطالمة يونان مصر سلال جين أوروبي الاخرى دخل مصر دخول روم مصر بناء تقدم مستغرب ظهر المومياوات صلى عمر قارب ألف ميلاد سلال جين ( j١ j٢ r ) جوار سلال جين محل e'

# input_ids = Tokenizer.encode(text, return_tensors="pt").to(device)
# summary_ids = Model.generate(input_ids, max_length=100, num_beams=4, decoder_start_token_id=Model.config.pad_token_id)

# summary = Tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)


In [ ]:
# # print(summary_ids)
# # print(input_ids)
# print(len(text))
# print(text)
# print(len(summary))
# print(summary)

1436
تاريخ أقول غزو " آشوريا " مصر حصل قرن سابع ميلاد احتال ثرا مصر مد تقارب المائة تخلل مقاوم حروب وقلاقل ثور مصري مقاوم احتلال الآشوري نهاية استطاع مصري طرد الآشوريين غزاة لبث غز فرس منتصف قرن سادس ميلاد سقط مصر يد فرس رزح احتلال فارس قراب ١٨٠ احتل الآشوريون مد تقارب قرن زم جاء يوناني قياد الإسكندر أكبر مقدوني حتل مصر وانهى حقب فارس مصر استمر حكم البطالمة مصر قراب قرون ما كاد مصر تخرج نفوذ حكم يونان وقع حكم رومان سمر سقط يد عمر العاص دخل مصر حكم عرب تخلل فتر حكم مصر خلال مماليك والاخشيديون أخير عثماني الأتراك خضع فترة قصير انتداب بريطاني نال استقلال تأسس جمهور مصر حديث واذا استثنى فترة انتداب بريطاني حكومة فتح الاسلامي مصر حكومة إسلام منذ فتح الاسلامي مصر توافد عرب زرافة مصر جزيرة عرب لاحظ تاريخ مصر سياسة حضار مدون بدأ تولي الاسرة فرعون ٣٥٠٠ ميلاد تعدي قدم حضار عالم تدوين حكام محكوم وقت افارقة محلي نقول عن اصحاب سلال جين e سلال جين لافارقة جنوب صحراء ( أمازيغ نوبي حبش ) انتهى حكم مصر خرج يد حكام محلي دخول الإسكندر أكبر مقدوني مصر ظن سلال جين j٢ دخل مصر قدوم الآشوريين فرس مصر دخل سلال

In [ ]:
# Train_Paragraphs[0]

'التاريخ يقول ان غزو " ا آشوريا " لمصر حصل في القرن السابع قبل الميلاد واحتلت على اثره مصر لمدة تقارب المائة سنة تخللها مقاومة وحروب وقلاقل وثورات من المصريين لمقاومة الاحتلال الآشوري وفي النهاية استطاع المصريون طرد الآشوريين الغزاة لكن ما لبثوا ان غزاهم الفرس في منتصف القرن السادس قبل الميلاد وسقطت مصر في أيدي الفرس ورزحت تحت الاحتلال الفارسي قرابة 180 سنة بعد ان احتلها الآشوريون لمدة تقارب قرنا من الزمان ثم جاء اليونانيون بقيادة الإسكندر الأكبر المقدوني واحتل مصر وانهى الحقبة الفارسية في مصر واستمر حكم البطالمة لمصر قرابة ثلاثة قرون وما كادت مصر تخرج من نفوذ الحكم اليوناني حتى وقعت تحت الحكم الروماني واسمر حتى سقط على يد عمرو بن العاص فدخلت مصر تحت حكم العرب الى اليوم وان كان تخلله فترات حكم مصر خلالها المماليك والاخشيديون وأخيرا العثمانيون الأتراك وخضعت فترة قصيرة للانتداب البريطاني حتى نالت استقلالها وتأسست جمهورية مصر الحديثة واذا استثنينا فترة الانتداب البريطاني فان كل الحكومات من بعد الفتح الاسلامي لمصر حكومات إسلامية .. ومنذ الفتح الاسلامي لمصر توافد العرب زرافات الى مصر من جزي

In [ ]:
# Scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

# Scores = []

# for predicted, test in zip(Predicted_Summaries, Test_Summaries):
#     score = Scorer.score(predicted, test)
#     Scores.append(score)

# Metrics = {
#     "rouge-1": [score["rouge1"].fmeasure for score in Scores],
#     "rouge-2": [score["rouge2"].fmeasure for score in Scores],
#     "rouge-l": [score["rougeL"].fmeasure for score in Scores]
# }

# print(Metrics)

In [ ]:
# Scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
# Scores = Scorer.score(Predicted_Summaries[0], Test_Summaries[0])

# Metrics = {
#     "rouge-1": Scores["rouge1"].fmeasure,
#     "rouge-2": Scores["rouge2"].fmeasure,
#     "rouge-l": Scores["rougeL"].fmeasure,
# }

# print(Metrics)

{'rouge-1': 0.0, 'rouge-2': 0.0, 'rouge-l': 0}


In [ ]:
# Test_Paragraphs[0]

'اعلنت " لجنة جائزة بشارة الخوري للتوعية الديموقراطية " انها انجزت المسابقة التي نظمتها حول " دور الرئيس بشارة الخوري في تحقيق الميثاق والاستقلال " بالتعاون مع وزارة التربية والتعليم العالي وبدعم من " مؤسسة فريديريتش ايبرت "، والتي شارك فيها طلاب الصف الثانوي الثاني بفرعيه العلمي والادبي والذين ينتمون الى مدارس رسمية وخاصة في المحافظات اللبنانية ، لافتة الى ان اعلان النتائج وتوزيع الجوائز سيتم خلال احتفال يقام عند الخامسة من بعد ظهر الخميس 30 الحالي في قصر الاونيسكو . واوضحت اللجنة في بيان " ان المسابقة تناولت تاريخ لبنان الحديث من قيام دولة لبنان الكبير في الاول من ايلول 1920، الى الجلاء في 31 كانون الاول 1946، دور الرئيس بشارة الخوري في تحقيق الوحدة الوطنية واهمية الالتزام بالميثاق الوطني للمحافظة على الاستقلال ". واشارت " الى ان الطلاب المشاركين اخضعوا في مرحلة اولى لامتحان خطي فاز في نهايته 69 طالبا ، ثم اشترك 17 منهم ممن نالوا العلامات الاعلى في المرحلة الثانية من المسابقة التي تضمنت امتحانا شفهيا امام لجنة فاحصة من الاساتذة المختصين اجابوا فيه على اسئلة تناولت فترة ولادة الاستقلا

In [ ]:
# Predicted_Summaries[0]

''

In [ ]:
# Test_Summaries[0]

'قالت لجنة جائزة بشارة الخوري أنها اجتازت المسابقة التي نظمتها حول بشارة الخوري في أن يحقق الاستقلال، وقام طلبة الصف الثاني الثانوي بالمشاركة في مؤسسة فريديريتش ايبريت، وأن توزيع الجوائز يتم في احتفال يحدث الخامسة بعد الظهر، واللجنة قالت في بيان أن المسابقة كانت تحتوي على تاريخ لبنان الحديث، وأن بشارة الخوري حافظ على الالتزام بالميثاق لكي ينال الاستقلال، وأعلن عن أسماء الطلبة الفائزين في احتفال الأونيسكو.'